In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import seml.database as db_utils
import jkutils.mpl_latex_template as mplt
import matplotlib
import os 
os.environ["PATH"] += os.pathsep + '/nfs/homedirs/schuchaj/texlive/2020/bin/x86_64-linux'

In [2]:
collection = 'cert_collective'
collection_train = 'train_collective'

jk_config = {
    'username': 'schuchaj',
    'password': 'LBGqsHEY',
    'host': 'fs.kdd.in.tum.de',
    'port': 27017,
    'db_name': 'schuchaj'
}

col = db_utils.get_collection(collection, mongodb_config=jk_config)
col_train = db_utils.get_collection(collection_train, mongodb_config=jk_config)

In [3]:
def find_items(dataset, model, max_rad, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None):
    
    db_filter = {
        'config.restrictions.dataset' : dataset,
        'config.restrictions.model' : model,
        'config.max_rad' : max_rad,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj,
        'config.pf_plus_att': pf_plus_att,
        'config.pf_minus_att': pf_minus_att,
        'config.local_budget_descriptor': None
    }
    if seed is not None:
        db_filter['config.seed'] = seed
    if col.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col.find(db_filter)
    
    ret = []
    
    
    for exp in exps:

        config = exp['config']
        if exp['result'] is None:
            continue
        cert_file_dir = exp['result']['cert_file']
        
        try:
            cert = torch.load(cert_file_dir)
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/cert_collective/{cert_file_dir.split("/")[-1]}')
            
        ret.append((exp, config, cert, exp['start_time'], exp['stop_time']))
            
    return ret

In [4]:
def find_items_train(dataset, model, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None):
    db_filter = {
        'config.dataset' : dataset,
        'config.model': model,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj
    }
    if seed is not None:
        db_filter['config.seed'] = seed

    if col_train.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col_train.find(db_filter)
    
    ret = []
    
    for exp in exps:
        config = exp['config']
        
        run_id = config['overwrite']
        db_collection = config['db_collection']
        save_dir = config['save_dir']

        try:
            cert = torch.load(f'{save_dir}/{db_collection}_{run_id}', map_location=torch.device('cpu'))
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/train_collective/{db_collection}_{run_id}', map_location=torch.device('cpu'))

        ret.append((exp, config, cert))
            
    return ret

In [5]:
def grid_from_exp(exp, collective=True):
    cert = exp[2]
    if collective:
        return cert['collective_grid']
    else:
        idx_test = cert['idx_test']
        return ((cert['grid_base'][idx_test] > 0.5).sum(axis=0))

In [6]:
def plot(grids, color, linestyle, x=None, label=None, plot_std=False, std_alpha=0.15, step=False, pad_last=False):
    mean = np.mean(grids, axis=0)
    
    if pad_last:
        mean = np.hstack([mean, [mean[-1]]])

    if x is None:
        x = np.arange(len(mean))
    
    if step:
        plt.step(x, mean, c=color, linestyle=linestyle, label=label, where='post', clip_on=False, zorder=3)
    else:
        plt.plot(x, mean, c=color, linestyle=linestyle, label=label, clip_on=False, zorder=3)
    
    if plot_std:
        std = np.std(grids, axis=0)
        if pad_last:
            std = np.hstack([std, [std[-1]]])

        if step:
            plt.fill_between(x, mean-std, mean+std, color=color, alpha=std_alpha, step='post')
        else:
            plt.fill_between(x, mean-std, mean+std, color=color, alpha=std_alpha)

In [7]:
# Styling
sns.set()
pal = sns.color_palette('colorblind', 3)

In [8]:
exps_collective_pubmeded = find_items('pubmed', 'GCN', [0, 0, -1], 0, 0.002, 0.6)
deltas = [
    8 * 60 * 60 + 32 * 60 + 28,
    8 * 60 * 60 + 17 * 60 + 23,
    8 * 60 * 60 + 41 * 60 + 36,
    8 * 60 * 60 + 53 * 60 + 6,
    8 * 60 * 60 + 3 * 60 + 21,
]
average_time = np.mean(np.array(deltas) / 101)
print(average_time)


302.7207920792079


In [9]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)
# Zoomed in
exps_collective_cora = find_items('cora_ml', 'GCN', [0, 0, -1], 0, 0.002, 0.6)
grids_cora = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_cora])
plot(grids_cora, pal[0], '-', label='Cora', plot_std=True, step=True, pad_last=True)

exps_collective_citeseer = find_items('citeseer', 'GCN', [0, 0, -1], 0, 0.002, 0.6)
grids_citeseer = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_citeseer])
plot(grids_citeseer, pal[1], '-', label='Citeseer', plot_std=True, step=True, pad_last=True)

exps_collective_pubmed = find_items('pubmed', 'GCN', [0, 0, -1], 0, 0.002, 0.6)
grids_pubmed = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_pubmed])
plot(grids_pubmed, pal[2], '-', label='Pubmed', plot_std=True, step=True, pad_last=True)

exps_train_cora = find_items_train('cora_ml', 'GCN', 0, 0.002, 0.6)
grids_train_cora = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_cora])
plot(grids_train_cora, pal[0], (0, (6, 2)),  plot_std=True, step=True)

exps_train_citeseer = find_items_train('citeseer', 'GCN', 0, 0.002, 0.6)
grids_train_citeseer = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_citeseer])
plot(grids_train_citeseer, pal[1], (0, (6, 2)),  plot_std=True, step=True)

exps_train_pubmed = find_items_train('pubmed', 'GCN', 0, 0.002, 0.6)
grids_train_pubmed = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_pubmed])
plot(grids_train_pubmed, pal[2], (0, (6, 2)),  plot_std=True, step=True)


plt.legend()
plt.xlabel('Attribute deletions')
plt.ylabel('Certified ratio')
plt.ylim(0, 1)
plt.xlim(0, 22)

mplt.savefig(f'../experiments/datasets/small_radii', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

In [10]:
print((grids_cora / grids_train_cora)[:, :-1].mean())
print((grids_citeseer / grids_train_citeseer)[:, :-1].mean())
print((grids_pubmed / grids_train_pubmed)[:, :-1].mean())

1.469414141808078
2.504658037415968
9.029135141030478


/nfs/homedirs/bojchevs/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/nfs/homedirs/bojchevs/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/nfs/homedirs/bojchevs/miniconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)

# Zoomed in
exps_collective_cora = find_items('cora_ml', 'GCN', [0, 0, 1000], 0, 0.002, 0.6)
grids_cora = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_cora])
plot(grids_cora, pal[0], '-', label='Cora', plot_std=True, x=np.arange(101) * 10)

exps_collective_citeseer = find_items('citeseer', 'GCN', [0, 0, 1000], 0, 0.002, 0.6)
grids_citeseer = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_citeseer])
plot(grids_citeseer, pal[1], '-', label='Citeseer', plot_std=True, x=np.arange(101) * 10)

exps_collective_pubmed = find_items('pubmed', 'GCN', [0, 0, 1000], 0, 0.002, 0.6)
grids_pubmed = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_pubmed])
plot(grids_pubmed, pal[2], '-', label='Pubmed', plot_std=True, x=np.arange(101) * 10)

exps_train_cora = find_items_train('cora_ml', 'GCN', 0, 0.002, 0.6)
grids_train_cora = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_cora])
plot(grids_train_cora, pal[0], (0, (6, 2)),  plot_std=True)

exps_train_citeseer = find_items_train('citeseer', 'GCN', 0, 0.002, 0.6)
grids_train_citeseer = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_citeseer])
plot(grids_train_citeseer, pal[1], (0, (6, 2)),  plot_std=True)

exps_train_pubmed = find_items_train('pubmed', 'GCN', 0, 0.002, 0.6)
grids_train_pubmed = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_pubmed])
plot(grids_train_pubmed, pal[2], (0, (6, 2)),  plot_std=True)


plt.legend()
plt.xlabel('Attribute deletions')
plt.ylabel('Certified ratio')
plt.ylim(0, 1)
plt.xlim(0, 1000)

mplt.savefig(f'../experiments/datasets/large_radii', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

In [14]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)
plot(grids_cora, pal[0], '-', label='Cora', plot_std=True, x=np.arange(101) * 10)
plot(grids_citeseer, pal[1], '-', label='Citeseer', plot_std=True, x=np.arange(101) * 10)
plot(grids_pubmed, pal[2], '-', label='Pubmed', plot_std=True, x=np.arange(101) * 10)

plot(grids_train_cora, pal[0], (0, (6, 2)),  plot_std=True, x=np.arange(22))
plot(grids_train_citeseer, pal[1], (0, (6, 2)),  plot_std=True, x=np.arange(22))
plot(grids_train_pubmed, pal[2], (0, (6, 2)),  plot_std=True, x=np.arange(22))

plt.legend(fancybox=False, frameon=True, loc='lower left')
plt.xlabel('Attribute deletions')
plt.ylabel('Certified ratio')
plt.ylim(0, 1)
plt.xlim(0, 1000)
plt.xscale('symlog')
mplt.savefig(f'../experiments/datasets/all_radii', format='pgf', preview='png', dpi=150, tight={'pad': 0.5})